In [ ]:
%load_ext autoreload
%autoreload 2


import sys
sys.path.append('../')
import os

In [42]:
import pandas as pd
import json

In [3]:
ratings_df = pd.read_csv("../data/flickr/processed/ratings.csv")

In [8]:
usr_threshold = 0

liked_counts = (
    ratings_df[ratings_df["score"] >= 4]
    .groupby("worker_id")["score"]
    .count()
    .reset_index(name="liked_count")
)

valid_users = liked_counts[liked_counts["liked_count"] >= usr_threshold]["worker_id"].unique()
valid_worker_id = liked_counts[liked_counts["liked_count"] >= usr_threshold]["worker_id"].unique()
filtered_ratings_df = ratings_df[ratings_df["worker_id"].isin(valid_users)].copy()

worker_mapping = {old_id: new_id for new_id, old_id in enumerate(valid_worker_id)}
filtered_ratings_df.rename(columns={"worker_id": "old_worker_id"}, inplace=True)
filtered_ratings_df["worker_id"] = filtered_ratings_df["old_worker_id"].map(worker_mapping)

In [9]:
filtered_ratings_df

,Unnamed: 0,worker,imagePair,score,old_worker_id,image_id,worker_id
0,0,A14W0IW2KGR80K,farm1_176_396297967_94dae8ad97.jpg,2,0,0,0
1,1,A14W0IW2KGR80K,farm1_106_268532564_c273eb23e6.jpg,4,0,1,0
2,2,A14W0IW2KGR80K,farm8_7224_7246491674_540646a594.jpg,4,0,2,0
3,3,A14W0IW2KGR80K,farm3_2643_4041742092_25f170c501.jpg,2,0,3,0
4,4,A14W0IW2KGR80K,farm4_3080_2922825219_1ba9c41b1a.jpg,5,0,4,0
...,...,...,...,...,...,...,...
193203,203099,A75N0N2PWNC9Q,farm8_7382_13133462963_5efb9fe9d4_b.jpg,5,209,38022,209
193204,203100,A75N0N2PWNC9Q,farm4_3577_3807605097_81a7d4da4e.jpg,3,209,38023,209
193205,203101,A75N0N2PWNC9Q,farm9_8493_8276626373_d72e32ca2d.jpg,4,209,38024,209
193206,203102,A75N0N2PWNC9Q,farm7_6158_6171447812_6426657080.jpg,4,209,38025,209


In [15]:
rec_data = pd.read_csv(f"../data/flickr/processed/train/train_usrthrs_{usr_threshold}.csv", )
image_captions = pd.read_csv(f"../data/flickr/processed/captions/image_captions.df")

In [39]:
metadata = []

for index, row in rec_data.iterrows():
    liked = row["score"] >= 4
    impath = row["imagePair"]
    worker_id = row["worker_id"]
    caption = image_captions.loc[image_captions["image"]==impath]["caption"].item()

    if liked:
        # assumes path starts from lora in ./data/flickr/preprocessed/lora
        metadata.append({
            'file_name': f"FLICKR-AES-001/40K/{impath}",
            'caption': f"<u{worker_id}> {caption}",
        })

In [ ]:
# Important to symlink images from raw
# ln -s /abs/path/to/FLICKR-AES-001 \ /home/gabriel/Projects/REBECA/data/flickr/processed/lora/FLICKR-AES-001

In [43]:
root = "/home/gabriel/Projects/REBECA/data/flickr/processed/lora"
out = os.path.join(root, "metadata.jsonl")
os.makedirs(root, exist_ok=True)
with open(out, "w", encoding="utf-8") as f:
    for r in metadata:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")